<h3> Importing </h3>

In [52]:
import pandas as pd
import xml.etree.ElementTree as et
import xmltodict
import glob
from datetime import datetime

<p> you also required two file paths that will be avalible globally in the code for all functions. these are <b> transformed_data.csv</b> to store the final output data that can be load to database and <b> log_file.txt</b> that stores all logs. </p>

In [53]:
target_file="transformed_data.csv"
log_file="log_file.txt"

<h2> Exract daat from multi sources </h2>

<h4>1. Extract data from csv file </h4>

In [54]:
def extract_from_csv(file_to_process):
    df=pd.read_csv(file_to_process)
    return df

<h4>2. Extract data from json file </h4>

In [55]:
def extract_from_json(file_to_process):
    try:
        df = pd.read_json(file_to_process, lines=True)
        return df
    except ValueError as e:
        return pd.DataFrame()


<h4>3. Extract from xml </h4>

In [56]:
def extract_from_xml(file_to_process):
   dataframe=pd.DataFrame(columns=['name','height','weight'])
   
   tree=et.parse(file_to_process)
   root=tree.getroot()
   for person in root:
      name=person.find("name").text
      height = float(person.find("height").text) 
      weight = float(person.find("weight").text) 
      dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
      return dataframe 

<p> create function to identify which function to call basis on file type of data file, which uses the glob method to identify file type </p>

In [57]:
def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data 
     
    # process all csv files 
    for csvfile in glob.glob("*.csv"): 
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True) 
         
    # process all json files 
    for jsonfile in glob.glob("*.json"): 
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True) 
     
    # process all xml files 
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True) 
         
    return extracted_data 

<h2> Transform data </h2>

In [58]:
def transform(data):
    '''Convert inches to meters and round off to two decimals 
    1 inch is 0.0254 meters '''
    
    data['height']=round(data.height*0.0254,2)
    
    '''Convert pounds to kilograms and round off to two decimals 
    1 pound is 0.45359237 kilograms '''
    data['weight']=round(data.weight* 0.45359237,2)
    
    return data

<h2> Loading and Logging </h2>

In [59]:
def load_data(target_file,transformed_data):
    transformed_data.to_csv(target_file)

<p>our need to implement the logging operation to record the progress of the different operations. For this operation, you need to record a message, along with its timestamp, in the log_file</p>

<p> to record a message, needed to implement a function log_progress() that accepts the log message as an argument , capture the current date and time using datetime function from datetime library.
</p>

In [60]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) #strftime attribute to convert the timestamp to a string format
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

In [61]:
# Log the initialization of the ETL process 
log_progress("ETL Job Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract() 
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_data(target_file,transformed_data) 
 
# Log the completion of the Loading process 
log_progress("Load phase Ended") 
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 

Transformed Data
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15  simon    1.72   50.97
16  simon    1.72   50.97
17  simon    1.72   50.97


C:\Users\GPU-TECH\AppData\Local\Temp\ipykernel_16772\442833592.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)
C:\Users\GPU-TECH\AppData\Local\Temp\ipykernel_16772\2648932179.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True)
C:\Users\GPU-TECH\AppData\Local\Temp\ipykernel_16772\26489